# 02 — Landmarks Analysis

This notebook helps you analyze **hand landmarks** detected by MediaPipe Tasks.

Goals:
- Print landmark coordinates for each hand
- Calculate distances and angles between points
- Visualize hands in real-time for debugging

Press **Q** to quit the camera feed.

### Imports

Libraries required for:
- Webcam access and visualization
- MediaPipe Tasks API (Hands Landmarker)
- Landmark geometry analysis

In [ ]:
import cv2
import time
from pathlib import Path

from mediapipe.tasks import python
from mediapipe.tasks.python import vision

import math
import mediapipe as mp


### Hand Landmarker setup

Downloads the MediaPipe hand landmark model if needed  
and initializes the Hands detector (Tasks API).

In [ ]:
MODEL_PATH = Path("hand_landmarker.task")

if not MODEL_PATH.exists():
    !wget -O hand_landmarker.task https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task
else:
    print("hand_landmarker.task already exists")

base_options = python.BaseOptions(
    model_asset_path=str(MODEL_PATH)
)

options = vision.HandLandmarkerOptions(
    base_options=base_options,
    num_hands=2,
    min_hand_detection_confidence=0.7,
    min_hand_presence_confidence=0.7
)

detector = vision.HandLandmarker.create_from_options(options)

print("MediaPipe Tasks (Hands) successfully initialized!")


### Real-time hand landmarks visualization and analysis

Captures the webcam stream, detects hand landmarks using MediaPipe Tasks,
draws landmark points and connections, and computes a simple pinch distance
(thumb tip ↔ index tip) for each detected hand.

In [ ]:
HAND_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 4),             # Thumb
    (0, 5), (5, 6), (6, 7), (7, 8),             # Index finger
    (9, 10), (10, 11), (11, 12),                # Middle finger
    (13, 14), (14, 15), (15, 16),               # Ring finger
    (0, 17), (17, 18), (18, 19), (19, 20),      # Pinky
    (5, 9), (9, 13), (13, 17)                   # Palm
]

# Computes the Euclidean distance between two landmarks
def get_distance(p1, p2):
    return math.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2)

# Camera initialization
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Cannot open camera")

last_print_time = 0

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to read frame")
            break

        # Mirror view for a more natural interaction
        frame = cv2.flip(frame, 1)
        h, w, _ = frame.shape

        # Convert frame to RGB (required by MediaPipe)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(
            image_format=mp.ImageFormat.SRGB,
            data=rgb_frame
        )

        # Hand landmark detection
        result = detector.detect(mp_image)

        if result.hand_landmarks:
            for hand_idx, landmarks in enumerate(result.hand_landmarks):
                
                # 1. Draw hand connections (skeleton)
                for start_idx, end_idx in HAND_CONNECTIONS:
                    start_pt = (
                        int(landmarks[start_idx].x * w),
                        int(landmarks[start_idx].y * h)
                    )
                    end_pt = (
                        int(landmarks[end_idx].x * w),
                        int(landmarks[end_idx].y * h)
                    )
                    cv2.line(frame, start_pt, end_pt, (0, 255, 0), 2)

                # 2. Draw landmark points
                for lm in landmarks:
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

                # 3. Simple analysis: pinch distance (thumb ↔ index)
                dist_pinch = get_distance(landmarks[4], landmarks[8])

                # Hand label: Left / Right
                hand_label = result.handedness[hand_idx][0].category_name

                # Display pinch distance on screen
                cv2.putText(
                    frame,
                    f"{hand_label} - Pinch: {dist_pinch:.2f}",
                    (10, 50 + hand_idx * 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (255, 0, 0),
                    2
                )

        # Display the camera feed
        cv2.imshow("MediaPipe Hands", frame)

        # Exit on 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()
    print("Camera closed cleanly")